# Practice with operations on Spark dataframe
Only use a fraction of all the availible data for faster speed, 100k lines.

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv('data/play/play.csv')

/Users/qingwang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.dtypes

uid           int64
device       object
song_id      object
play_time    object
song_len     object
date         object
dtype: object

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

uid          0
device       0
song_id      0
play_time    0
song_len     0
date         0
dtype: int64

In [12]:
df.groupby(['date']).count()

,uid,device,song_id,play_time,song_len
date,,,,,
20170330,287766,287766,287766,287766,287766
20170331,554875,554875,554875,554875,554875
20170401,475218,475218,475218,475218,475218
20170402,446415,446415,446415,446415,446415
20170403,233073,233073,233073,233073,233073
20170404,336983,336983,336983,336983,336983
20170405,274091,274091,274091,274091,274091
20170406,276100,276100,276100,276100,276100
20170407,256903,256903,256903,256903,256903


In [13]:
df = df[df['date']!='238']
df = df[df['date']!='None']
df.to_csv('data/play/play_tmp.csv', index=False)

In [14]:
print('# of records: ', df.shape[0])

# of records:  9462209


In [39]:
chosen_rows = random.sample(list(np.arange(0, df.shape[0])), 100000)

In [40]:
df.iloc[chosen_rows, ].to_csv('data/event_100k.csv', index=False)

## Create a SparkSession and load the csv file.

Also import the sql functions to do the selections, groups ....sql like operations.

In [41]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [42]:
spark = SparkSession \
        .builder \
        .appName('Practice on Spark dataframe') \
        .getOrCreate()

In [43]:
df = spark.read.csv('data/event_100k.csv', header=True).cache()

In [44]:
df.show(5)

+---------+-----+----------+--------+
|      uid|event|   song_id|    date|
+---------+-----+----------+--------+
|168033234|    P|   7169674|20170410|
|168165789|    D|  663856.0|20170406|
|168667361|    P|       0.0|20170417|
|168327828|    P|   2597849|20170425|
|168275357|    D|22845435.0|20170409|
+---------+-----+----------+--------+
only showing top 5 rows



In [45]:
df.select('uid').distinct().count()

18645

In [46]:
df.groupby('date').count().show()

+--------+-----+
|    date|count|
+--------+-----+
|20170504| 1639|
|20170413| 2507|
|20170410| 2732|
|20170428| 1903|
|20170417| 2159|
|20170420| 2144|
|    None|  101|
|20170430| 1927|
|20170405| 3106|
|20170505| 1663|
|20170404| 3715|
|       0|    2|
|20170402| 5080|
|20170424| 1148|
|20170406| 2966|
|20170416| 2446|
|20170331| 3334|
|20170429| 1975|
|20170415| 2396|
|20170419| 2127|
+--------+-----+
only showing top 20 rows

